## Backtesting execution

In [35]:
# os.chdir(r"C:\Users\SamuliMustonen\Documents\Ready Solutions\Docs\Muut\DataAnalysis\Investing\models")

In [ ]:
print(os.getcwd())

In [1]:
import requests
import pandas as pd
# from datetime import datetime, timedelta, date
from datetime import datetime
import time
from polygon import RESTClient
import logging
import signal
import sys
import pickle
import lz4.frame  # type: ignore
import concurrent.futures
import os
import sys
import pandas as pd
import numpy as np
import glob
import nbimporter
import gzip
from modelPEG import main


In [ ]:
# Call the main function and store the returned DataFrame
df = main()

In [ ]:
# Show dataframe
print(df.head())

## Backtesting model 1
Condition 1: The portfolio cash must be at least 1,000 for a buy to occur. 
Condition 2: You can only hold shares of one ticker at a time (current_symbol keeps track of the current stock).
Condition 3: The backtest enforces a rule where a new buy cannot occur until the previous position is sold. The next trade can only happen on or after the sell date.
Condition 4: The backtesting runs until the data is exhausted (the last date in the dataset).


In [15]:
# Model 1

# Example DataFrame setup
# df = pd.DataFrame({
#     'timestamp': pd.to_datetime([1,2,3,4,5,6,7,8,9,10], unit='D', origin='2022-01-01'),  # sample dates
#     'symbol': ['AA','AA','AA','AA','AA','AA','AA','AA','AA','AA'],  
#     'close': [10,11,11,12.5,13,12,12,14.5,13,15],
#     'entryPos': [0,1,0,0,0,1,0,0,0,0],  
#     'exitPos': [0,0,0,1,0,0,0,1,0,0],
#     'volatility': [0.03,0.03,0.03,0.03,0.03]*2
# })

# Initial setup
initial_cash = 10000
portfolio_cash = initial_cash
portfolio_stock = 0  # initially no stock
current_symbol = None  # track current symbol in the portfolio
buy_price = 0
trade_fee_percent = 0.00  # 0.1%
stop_loss = 1 - (2 * df['volatility'])  # Stop loss calculated by 1 - (2 * volatility)
risk = 0.05 # How many percentage risked for each trade from portfolio value. 0.05 = 5%.

# Metrics initialization
total_trades = 0
total_profit = 0
max_profit = 0
max_drawdown = 0
peak_value = initial_cash
total_fees = 0

# Trade records
trades = []
win_trades = 0
trade_durations = []

# Condition constants
min_cash_to_trade = 1000
last_trade_date = None  # track the date of the last trade (sell)

# Buy condition function
def buy_condition(entryPos, portfolio_cash, portfolio_stock, current_symbol, last_trade_date, current_date):
    return entryPos == 1 and portfolio_cash >= min_cash_to_trade and portfolio_stock == 0 and \
           (last_trade_date is None or current_date >= last_trade_date) and current_symbol is None

# Sell condition function
def sell_condition(close_price, exitPos, buy_price, stop_loss):
    sell_condition_profit = exitPos == 1
    sell_condition_loss = close_price <= buy_price * stop_loss
    return sell_condition_profit or sell_condition_loss

# Backtesting function
def backtest(df):
    global portfolio_cash, portfolio_stock, buy_price, total_trades, total_profit, max_profit, max_drawdown, peak_value, current_symbol, last_trade_date, total_fees, win_trades, trade_durations
    
    for index, row in df.iterrows():
        close_price = row['close']
        entryPos = row['entryPos']
        exitPos = row['exitPos']
        ticker = row['symbol']
        current_date = row['timestamp']
        volatility = row['volatility']
        
        # Calculate stop loss for the current row
        current_stop_loss = 1 - (2 * row['volatility'])
        
        # Sell condition (sell only if there's stock)
        if portfolio_stock > 0 and sell_condition(close_price, exitPos, buy_price, current_stop_loss):
            total_value = portfolio_stock * close_price
            fee = total_value * trade_fee_percent
            total_value -= fee  # after fee
            total_fees += fee  # accumulate total fees
            
            # Sell the stock
            profit = total_value - (portfolio_stock * buy_price) - (portfolio_stock * buy_price * trade_fee_percent)
            portfolio_cash += total_value
            total_profit += profit
            trade_duration = (current_date - last_trade_date).days  # Track trade duration
            trade_durations.append(trade_duration)
            
            if profit > 0:
                win_trades += 1
            
            portfolio_stock = 0
            last_trade_date = current_date
            current_symbol = None
            
            trades.append({'date': current_date, 'symbol': ticker, 'type': 'sell', 'price': close_price, 'shares': portfolio_stock, 'fee': fee, 'profit': profit, 'duration': trade_duration})
            
            # Calculate max profit and drawdown
            current_value = portfolio_cash
            if current_value > peak_value:
                peak_value = current_value
            drawdown = (peak_value - current_value) / peak_value
            max_drawdown = max(max_drawdown, drawdown)
            max_profit = max(max_profit, profit)
        
        # Buy condition
        if buy_condition(entryPos, portfolio_cash, portfolio_stock, current_symbol, last_trade_date, current_date):
            position_size = (risk * portfolio_cash) / (2 * volatility) # THIS IS MANUALLY ADDED
            print(f'portfolio cash after sell + liq: {portfolio_cash}')
            print(f'next position size: {position_size}')
            shares_to_buy = int(position_size / close_price)
            total_cost = shares_to_buy * close_price
            fee = total_cost * trade_fee_percent
            total_cost += fee  # including fee
            total_fees += fee  # accumulate total fees
            
            # Buy the stock
            portfolio_stock += shares_to_buy
            portfolio_cash -= total_cost
            buy_price = close_price
            current_symbol = ticker
            last_trade_date = current_date
            
            trades.append({'date': current_date, 'symbol': ticker, 'type': 'buy', 'price': close_price, 'shares': shares_to_buy, 'fee': fee})
            total_trades += 1
            print(f'Cash left after buy: {portfolio_cash}')
            
            
    
    # Calculate final portfolio value and returns
    final_portfolio_value = portfolio_cash
    total_return = round((final_portfolio_value - initial_cash) / initial_cash * 100, 2)  # in %
    
    # Calculate win rate
    win_rate = round(win_trades / total_trades * 100, 2) if total_trades > 0 else 0
    
    # Calculate best and worst trades (returns in %)
    trade_returns = [(t['profit'] / (t['shares'] * buy_price)) * 100 
                 for t in trades if t['type'] == 'sell' and t['shares'] > 0 and buy_price > 0]
    best_trade = round(max(trade_returns, default=0), 2)
    worst_trade = round(min(trade_returns, default=0), 2)
    
    # Average trade duration
    avg_duration = round(sum(trade_durations) / len(trade_durations), 2) if trade_durations else 0
    
    return {
        'trades': trades,
        'final_portfolio_value': round(final_portfolio_value, 2),
        'total_return': total_return,
        'total_profit': round(total_profit, 2),
        'total_trades': total_trades,
        'total_fees': round(total_fees, 2),
        'win_rate': win_rate,
        'best_trade': best_trade,
        'worst_trade': worst_trade,
        'avg_trade_duration': avg_duration,
        'max_profit': round(max_profit, 2),
        'max_drawdown': round(max_drawdown * 100, 2)  # in %
    }

# Run backtest
results = backtest(df)

# Convert trades to DataFrame for analysis
trades_df = pd.DataFrame(results['trades'])

# Output results
print(f"Final portfolio cash: {results['final_portfolio_value']}")
print(f"Total return: {results['total_return']}%")
print(f"Total profit: {results['total_profit']}")
print(f"Total trades: {results['total_trades']}")
print(f"Total fees: {results['total_fees']}")
print(f"Win rate: {results['win_rate']}%")
print(f"Best trade: {results['best_trade']}%")
print(f"Worst trade: {results['worst_trade']}%")
print(f"Average trade duration: {results['avg_trade_duration']} days")
print(f"Max profit from a single trade: {results['max_profit']}")
print(f"Max drawdown: {results['max_drawdown']}%")

portfolio cash after sell + liq: 10000
next position size: 8333.333333333334
Cash left after buy: 1673.0
portfolio cash after sell + liq: 11135.5
next position size: 9279.583333333334
Cash left after buy: 1859.5
Final portfolio cash: 13068.0
Total return: 30.68%
Total profit: 3068.0
Total trades: 2
Total fees: 0.0
Win rate: 100.0%
Best trade: 0%
Worst trade: 0%
Average trade duration: 2.0 days
Max profit from a single trade: 1932.5
Max drawdown: 0%


In [14]:
trades_df.head()

,date,symbol,type,price,shares,fee,profit,duration
0,2022-01-03,AA,buy,11.0,757,0.0,NaN,NaN
1,2022-01-05,AA,sell,12.5,0,0.0,1135.5,2.0
2,2022-01-07,AA,buy,12.0,773,0.0,NaN,NaN
3,2022-01-09,AA,sell,14.5,0,0.0,1932.5,2.0


In [ ]:
# Iterate the backtesting for each symbol in dataframe

# List of symbols to iterate over
symbols = df['symbol'].unique()

# Initialize a list to collect results for each symbol
all_results = []
all_trades = []

for symbol in symbols:
    # Filter the DataFrame to only include data for the current symbol
    df_symbol = df[df['symbol'] == symbol]
    
    # Run backtest for the current symbol
    result = backtest(df_symbol)
    
    # Store the result in the list
    all_results.append({
        'symbol': symbol,
        'final_portfolio_value': result['final_portfolio_value'],
        'total_return': result['total_return'],
        'total_profit': result['total_profit'],
        'total_trades': result['total_trades'],
        'total_fees': result['total_fees'],
        'win_rate': result['win_rate'],
        'best_trade': result['best_trade'],
        'worst_trade': result['worst_trade'],
        'avg_trade_duration': result['avg_trade_duration'],
        'max_profit': result['max_profit'],
        'max_drawdown': result['max_drawdown']
    })
    
    # Store trade details separately for detailed analysis
    trades_df = pd.DataFrame(result['trades'])
    trades_df['symbol'] = symbol  # Add symbol to each trade for identification
    all_trades.append(trades_df)

# Combine all trades data into one DataFrame
combined_trades_df = pd.concat(all_trades, ignore_index=True)

# Convert results to a DataFrame for easier analysis
combined_results_df = pd.DataFrame(all_results)

# Output combined results
print(combined_results_df)
print(combined_trades_df)
